In [8]:
from tokenizer import Tokenizer
import pandas as pd
from IPython.display import display, Math

# Tokenizer

In [ ]:
df = pd.read_csv('../data/im2latex_formulas.norm.csv')
formulas = df['formulas'].tolist()

In [ ]:
tokenizer = Tokenizer()
tokenizer.build_vocab(formulas)
print('Vocab size:', tokenizer.vocab_size)

Vocab size: 532


In [10]:
s = formulas[0]
ids = tokenizer.encode(s)
display('Original seq', Math(s))
print('ids:', ids)
display('Decoded seq', Math(tokenizer.decode(ids)))

'Original seq'

<IPython.core.display.Math object>

ids: [1, 70, 6, 5, 12, 100, 4, 7, 5, 118, 4, 24, 40, 159, 59, 5, 29, 4, 7, 5, 12, 40, 146, 4, 70, 6, 5, 12, 100, 4, 7, 5, 118, 4, 24, 40, 7, 5, 48, 4, 59, 5, 29, 4, 7, 5, 12, 40, 7, 5, 48, 4, 146, 4, 40, 40, 7, 5, 48, 4, 5, 16, 5, 40, 7, 5, 48, 4, 12, 40, 4, 5, 40, 15, 40, 7, 5, 48, 4, 4, 4, 113, 35, 25, 64, 7, 5, 48, 48, 4, 8, 40, 9, 12, 5, 16, 5, 35, 4, 5, 44, 4, 4, 36, 25, 64, 8, 40, 9, 119, 13, 19, 23, 2]


'Decoded seq'

<IPython.core.display.Math object>